##  Preparando o Ambiente
### Importando as Bibliotecas Essenciais

In [1]:
import pandas             as pd
import numpy              as np
from plyer import notification 
from datetime import datetime
import requests
import os
import zipfile

# Baixando e Extraindo os Dados

Neste módulo, focaremos em obter os dados que serão utilizados na nossa análise. Implementaremos uma função para baixar um arquivo ZIP de uma URL específica e extrair seu conteúdo para o diretório atual.

In [3]:
def baixar_e_extrair(url, nome_arquivo):
  
  # Verifica se o arquivo já existe
  if os.path.exists(nome_arquivo):
    print(f"O arquivo {nome_arquivo} já existe. Pulando o download.")
    return

  # Baixa o arquivo
  response = requests.get(url)
  response.raise_for_status()

  # Salva o arquivo no disco
  with open(nome_arquivo, 'wb') as f:
    f.write(response.content)

  # Extrai o arquivo ZIP
  with zipfile.ZipFile(nome_arquivo, 'r') as zip_ref:
    zip_ref.extractall()

# URL do arquivo ZIP:
url = "https://download.inep.gov.br/microdados/microdados_enem_2023.zip"  
nome_arquivo = "microdados_enem_2023.zip"  # Nome do arquivo baixado

# Chama a função para baixar e extrair o arquivo
baixar_e_extrair(url, nome_arquivo)

O arquivo microdados_enem_2023.zip já existe. Pulando o download.


## Sistema de Alerta 

**Objetivo:**

Criar um sistema de notificações para alertar o usuário sobre erros ou eventos importantes durante a execução do programa.

**Função `alerta`:**

Esta função gera e envia notificações personalizadas ao sistema operacional, utilizando a biblioteca `plyer`.


In [4]:
def alerta(nivel, base, etapa, erro=""):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nível", nivel, "não disponível!")
        return

    # Enviar notificação
    notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
    )

## Carregando Arquivos CSV Grandes em Partes

**Objetivo:**

Carregar arquivos CSV muito grandes em partes menores para evitar problemas de memória e melhorar o desempenho.


In [1]:
import pandas as pd

def carregar_csv_em_partes(caminho_csv, chunksize=10000):
    for chunk in pd.read_csv(caminho_csv, chunksize=chunksize, encoding='latin1', sep=';'):
        yield chunk

# Caminho para o arquivo CSV
caminho_csv = "DADOS/MICRODADOS_ENEM_2023.csv"

# Processa o arquivo em partes de 10.000 linhas
for chunk in carregar_csv_em_partes(caminho_csv):
    # Processar cada chunk aqui
    print(chunk.head())  # Exemplo: imprimir as primeiras 5 linhas de cada chunk

   NU_INSCRICAO  NU_ANO  TP_FAIXA_ETARIA TP_SEXO  TP_ESTADO_CIVIL  \
0  210059085136    2023               14       M                2   
1  210059527735    2023               12       M                2   
2  210061103945    2023                6       F                1   
3  210060214087    2023                2       F                1   
4  210059980948    2023                3       F                1   

   TP_COR_RACA  TP_NACIONALIDADE  TP_ST_CONCLUSAO  TP_ANO_CONCLUIU  TP_ESCOLA  \
0            1                 1                1               17          1   
1            1                 0                1               16          1   
2            1                 1                1                0          1   
3            3                 1                2                0          2   
4            3                 1                2                0          2   

   ...  Q016  Q017  Q018 Q019  Q020 Q021  Q022  Q023  Q024  Q025  
0  ...     C     C     B    B  

## Análise Exploratória do DataFrame

**Visualizando as informações do DataFrame**


In [7]:
df = chunk
df.info

<bound method DataFrame.info of          NU_INSCRICAO  NU_ANO  TP_FAIXA_ETARIA TP_SEXO  TP_ESTADO_CIVIL  \
3930000  210059112392    2023                4       M                1   
3930001  210060912157    2023                5       F                1   
3930002  210059585519    2023                1       M                1   
3930003  210058919810    2023                9       M                2   
3930004  210059022001    2023                9       F                1   
...               ...     ...              ...     ...              ...   
3933950  210061959676    2023               12       M                1   
3933951  210061950911    2023                1       F                1   
3933952  210061965966    2023                3       F                1   
3933953  210061932304    2023                2       M                1   
3933954  210058924455    2023                3       F                1   

         TP_COR_RACA  TP_NACIONALIDADE  TP_ST_CONCLUSAO  TP_ANO_CON

## Obtendo as Dimensões do DataFrame

In [8]:
df.shape

(3955, 76)

## Acessando os Nomes das Colunas de um DataFrame

In [9]:
df.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',

## Análise de Dados Ausentes

**Objetivo:**

Identificar e quantificar os valores ausentes (NaN) em cada coluna do DataFrame.


In [10]:
dados_ausentes = df.isna().sum()
dados_ausentes

NU_INSCRICAO       0
NU_ANO             0
TP_FAIXA_ETARIA    0
TP_SEXO            0
TP_ESTADO_CIVIL    0
                  ..
Q021               0
Q022               0
Q023               0
Q024               0
Q025               0
Length: 76, dtype: int64

## Função `calcular_percentual_missing`

**Objetivo:**
Calcular o percentual de valores ausentes (NaN) em cada coluna de um DataFrame do Pandas.

In [11]:
def calcular_percentual_missing(df):
    # Calculando o percentual de valores ausentes
    percentual_missing = df.isna().sum() / len(df) * 100
    
    # Ordenando os resultados em ordem decrescente
    percentual_missing = percentual_missing.sort_values(ascending=False)
    
    return percentual_missing

In [12]:
calcular_percentual_missing(df)

TP_LOCALIZACAO_ESC        84.247788
CO_UF_ESC                 84.247788
TP_SIT_FUNC_ESC           84.247788
TP_DEPENDENCIA_ADM_ESC    84.247788
SG_UF_ESC                 84.247788
                            ...    
TP_PRESENCA_LC             0.000000
TP_PRESENCA_MT             0.000000
NU_ANO                     0.000000
TP_LINGUA                  0.000000
Q025                       0.000000
Length: 76, dtype: float64

In [13]:
df.index

RangeIndex(start=3930000, stop=3933955, step=1)

## Analisando a Variabilidade dos Dados

**Objetivo:**
Identificar as colunas com menor variabilidade de valores únicos e suas respectivas quantidades.

In [14]:
df.nunique().sort_values(ascending=True)

NU_ANO                1
Q025                  2
Q020                  2
Q018                  2
TP_LINGUA             2
                   ... 
TX_RESPOSTAS_MT    1844
TX_RESPOSTAS_CN    1844
TX_RESPOSTAS_LC    2027
TX_RESPOSTAS_CH    2027
NU_INSCRICAO       3955
Length: 76, dtype: int64

In [15]:
type(df)

pandas.core.frame.DataFrame

## Análise Descritiva Sumarizada

**Objetivo:**
Obter um resumo conciso das estatísticas descritivas de todas as colunas numéricas do DataFrame.

In [16]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NU_INSCRICAO,3955.0,2.100603e+11,1.092976e+06,2.100581e+11,2.100594e+11,2.100600e+11,2.100615e+11,2.100621e+11
NU_ANO,3955.0,2.023000e+03,0.000000e+00,2.023000e+03,2.023000e+03,2.023000e+03,2.023000e+03,2.023000e+03
TP_FAIXA_ETARIA,3955.0,6.839949e+00,4.455640e+00,1.000000e+00,3.000000e+00,6.000000e+00,1.100000e+01,1.900000e+01
TP_ESTADO_CIVIL,3955.0,1.105942e+00,5.261523e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00
TP_COR_RACA,3955.0,2.022503e+00,9.945473e-01,0.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,5.000000e+00
TP_NACIONALIDADE,3955.0,1.030341e+00,2.306547e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00
TP_ST_CONCLUSAO,3955.0,1.495575e+00,7.399081e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,4.000000e+00
TP_ANO_CONCLUIU,3955.0,4.403540e+00,5.767497e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.000000e+00,1.700000e+01
TP_ESCOLA,3955.0,1.235398e+00,4.553521e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
TP_ENSINO,790.0,1.003797e+00,6.154545e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00


## Preparando os Dados para Análise da Nota de Matemática

**Objetivo:**
Isolar as colunas relevantes para a análise de matemática e remover linhas com dados faltantes.


In [17]:
def encontrar_dataframe_matematica(df):
    # Filtrar as colunas desejadas
    df_matematica = df[['TP_FAIXA_ETARIA', 'TP_SEXO', 'CO_PROVA_MT']]
    
    # Remover linhas com valores NaN
    df_matematica_limpa = df_matematica.dropna()
    
    return df_matematica_limpa

In [18]:
encontrar_dataframe_matematica(df)

,TP_FAIXA_ETARIA,TP_SEXO,CO_PROVA_MT
3930000,4,M,1213.0
3930001,5,F,1213.0
3930002,1,M,1211.0
3930005,3,F,1214.0
3930006,1,F,1214.0
...,...,...,...
3933946,12,F,1212.0
3933948,3,M,1214.0
3933950,12,M,1211.0
3933951,1,F,1211.0


In [19]:
df_matematica_limpa = encontrar_dataframe_matematica(df)

In [20]:
df_matematica_limpa.shape

(1871, 3)

## Análise Descritiva das Notas de Matemática

**Objetivo:**
Obter um resumo estatístico das notas de matemática, considerando as variáveis de faixa etária e sexo.

In [21]:
df_matematica_limpa.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TP_FAIXA_ETARIA,1871.0,6.193479,4.523968,1.0,2.0,4.0,11.0,19.0
CO_PROVA_MT,1871.0,1212.874399,5.624121,1211.0,1212.0,1212.0,1213.0,1294.0


## Análise da Distribuição de Gênero na Prova de Matemática

**Objetivo:**
Contabilizar a quantidade de participantes de cada gênero na prova de matemática.

In [22]:
df_matematica_limpa.TP_SEXO.value_counts()

TP_SEXO
M    979
F    892
Name: count, dtype: int64